## Modules

In [16]:
"""
Private
"""


class KI_PRIVATE:
    @staticmethod
    def get_app_key():
        app_key = "your_app_key"
        return app_key

    @staticmethod
    def get_app_secret():
        app_secret = "your_app_secret"
        return app_secret

    @staticmethod
    def get_acc_number():
        acc_number = "your_acc_number"
        return acc_number


class DART_PRIVATE:
    @staticmethod
    def get_api_key():
        api_key = "your_api_key"
        return api_key


from private import KI_PRIVATE, DART_PRIVATE

In [17]:
"""
Broker
"""


class DART_BROKER:
    def __init__(self, dart_private) -> None:
        self.dart_private = dart_private

    def __call__(self):
        import dart_fss as dart_broker

        dart_broker.set_api_key(
            api_key=self.dart_private.get_api_key(),
        )
        return dart_broker


class KI_BROKER:
    def __init__(self, ki_private) -> None:
        self.ki_private = ki_private

    def __call__(self):
        from mojito import KoreaInvestment

        ki_broker = KoreaInvestment(
            api_key=self.ki_private.get_app_key(),
            api_secret=self.ki_private.get_app_secret(),
            acc_no=self.ki_private.get_acc_number(),
        )
        return ki_broker

In [18]:
import pandas as pd
import datetime as dt
from requests import request


class KI_LOADER:
    def __init__(self, ki_broker) -> None:
        self.ki_broker = ki_broker

    def load_stockcodes(self):
        symbols_df = self.ki_broker.fetch_symbols()
        stockcode_list = sorted(set(symbols_df["단축코드"]))
        return stockcode_list

    def load_ohclv_df(self, stockcode, ticks):
        tick_list = self._calculate_tick_list(ticks)
        ohclv_df_list = list()
        for tick in tick_list:
            start_day, end_day = self._calculate_date(tick)
            single_ohlcv_df = self._load_single_ohlcv_df(
                self.ki_broker, stockcode, start_day, end_day
            )
            ohclv_df_list.append(single_ohlcv_df)
        ohclv_df = pd.concat(ohclv_df_list)
        ohclv_df["StockCode"] = stockcode
        return ohclv_df

    def load_total_ohlcv_df(self, stockcodes, ticks):
        total_ohclv_df_list = list()
        for stockcode in stockcodes:
            try:
                ohclv_df = self.load_ohclv_df(stockcode, ticks)
                total_ohclv_df_list.append(ohclv_df)
            except:
                pass
        total_ohclv_df = pd.concat(total_ohclv_df_list)
        return total_ohclv_df

    def load_info_dict(self, stockcode):
        info_dict = self._load_info_dict(self.ki_broker, stockcode)
        return info_dict

    def load_total_info_df(self, stockcodes):
        total_info_dict_list = list()
        for stockcode in stockcodes:
            info_dict = self.load_info_dict(stockcode)
            total_info_dict_list.append(info_dict)
        total_info_df = pd.DataFrame(total_info_dict_list)
        return total_info_df

    @staticmethod
    def _calculate_tick_list(ticks):
        tick_list = [(i * 100, (i * 100) + 100) for i in range(ticks // 100)]
        last_range = ((ticks // 100) * 100, ((ticks // 100) * 100 + ticks % 100))
        tick_list.append(last_range)
        return tick_list

    @staticmethod
    def _calculate_date(tick):
        start_day = (dt.date.today() - dt.timedelta(days=tick[1])).strftime("%Y%m%d")
        end_day = (dt.date.today() - dt.timedelta(days=tick[0])).strftime("%Y%m%d")
        return (start_day, end_day)

    @staticmethod
    def _load_single_ohlcv_df(ki_broker, stockcode, start_day, end_day):
        ohlcv_resp = ki_broker.fetch_ohlcv(
            symbol=stockcode,
            timeframe="D",
            adj_price=True,
            start_day=start_day,
            end_day=end_day,
        )
        single_ohlcv_df = pd.DataFrame(ohlcv_resp["output2"])
        return single_ohlcv_df

    @staticmethod
    def _load_info_dict(ki_broker, stockcode):
        ohlcv_resp = ki_broker.fetch_ohlcv(symbol=stockcode)
        info_dict = ohlcv_resp["output1"]
        return info_dict


class DART_LOADER:
    def __init__(self, dart_broker) -> None:
        self.dart_broker = dart_broker

    def load_total_fundamental_df(self, stockcodes):
        corpcode_df = self._load_corpcode_df(self.dart_broker)
        corpcodes = self._get_corpcodes(corpcode_df, stockcodes)
        corpcode_zips = self._get_chunked_list(corpcodes, 99)

        fundamental_df_list = list()
        for corpcode_zip in corpcode_zips:
            try:
                _fundamental_df = self._load_fundamental_df(
                    corpcode_zip, self.dart_broker.get_api_key()
                )
                fundamental_df_list.append(_fundamental_df)
            except:
                pass
        fundamental_df = pd.concat(fundamental_df_list, axis=0)

        return fundamental_df

    @staticmethod
    def _load_corpcode_df(dart_broker):
        corpcode_dict = dart_broker.api.filings.get_corp_code()
        corpcode_df = pd.DataFrame(corpcode_dict)
        return corpcode_df

    @staticmethod
    def _get_corpcodes(corpcode_df, stockcodes):
        corpcodes = sorted(
            set(corpcode_df[corpcode_df["stock_code"].isin(stockcodes)]["corp_code"])
        )
        return corpcodes

    @staticmethod
    def _get_chunked_list(_list, n):
        chunked_list = [_list[i : i + n] for i in range(0, len(_list), n)]
        return chunked_list

    @staticmethod
    def _load_fundamental_df(corp_code_zip, api_key, year=2023, reprt_code=11013):
        url = f"https://opendart.fss.or.kr/api/fnlttMultiAcnt.json?crtfc_key={api_key}&corp_code={','.join(corp_code_zip)}&bsns_year={year}&reprt_code={reprt_code}"
        result = request("get", url)
        fundamental_df = pd.DataFrame(result.json()["list"])
        return fundamental_df

## Codes

In [19]:
# Private
ki_private = KI_PRIVATE()
dart_private = DART_PRIVATE()

In [20]:
# Brokers
ki_broker = KI_BROKER(ki_private)()
dart_broker = DART_BROKER(dart_private)()

In [21]:
# Loaders
ki_loader = KI_LOADER(ki_broker)
dart_loader = DART_LOADER(dart_broker)

In [22]:
# Independent load
stockcodes = ki_loader.load_stockcodes()

In [23]:
# Dependent load
total_fundamental_df = dart_loader.load_total_fundamental_df(stockcodes)
total_info_df = ki_loader.load_total_info_df(stockcodes)
total_ohlcv_df = ki_loader.load_total_ohlcv_df(stockcodes, 100)

Output()

In [24]:
# Dynamic Data
## Not Locally saved in production
total_fundamental_df.to_csv("./data/dynamic/total_fundamental_df.csv")
total_info_df.to_csv("./data/dynamic/total_info_df.csv")
total_ohlcv_df.to_csv("./data/dynamic/total_ohlcv_df.csv")